In [1]:
## 일반 패키지 ##
import sys
import time
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import sys
import time
import pandas as pd
import numpy as np
import os
from math import log
import warnings
warnings.filterwarnings(action='ignore')

### 워드클라우드 및 시각화 관련 패키지 ###
from collections import defaultdict
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

# 말뭉치 분석 및 명사 추출 패키지 #
from konlpy.tag import Kkma       ; kkma = Kkma()
from konlpy.tag import Hannanum   ; hannanum = Hannanum()
from konlpy.tag import Okt        ; t = Okt()
from konlpy.tag import *
import pickle

## 텍스트 통계분석 패키지 ##
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA


## word2vec 관련 패키지 ##
import gensim
from gensim.models import Word2Vec


from apyori import apriori
import networkx as nx
from collections import Counter
from PIL import Image

# Keep in mind the background of the image used must be white



# 그림 옵션 지정 부분 #
plt.rcParams ['font.family'] = [ 'sans-serif']
plt.rcParams ['font.sans-serif'] = ['Malgun Gothic']
plt.rcParams["figure.figsize"] = (10,10)
plt.rcParams['axes.grid'] = False
csfont = {'fontname':'Malgun Gothic'}
# 그림 옵션 지정 부분 #

In [ ]:
keyword = ['RE.csv']
stopwords = pd.read_csv("./stopwords.txt",encoding='utf-')
stopwordsList = list(stopwords.stopwords)

for k in keyword:

    print(k)
    # 파일을 읽기
    data = pd.read_csv('./'+k+'.csv')
    print(data)

    # 단어가 없는 경우 넘어가기 위한 장치
    def isNaN(string):
        return string != string

    # 리뷰 파일로부터 한국어의 명사 추출 (이는 TF-IDF를 구하기 전 전처리를 위함)
    content = data['Content']

    content_result = []
    count = -1
    for i in content:
        check = isNaN(i)
        if check == True:
            continue

        content_part = kkma.nouns(i) # 각 내용으로부터 명사 추출

        for j in content_part:

            if(len(j) == 1):
                continue
            elif(j in stopwordsList):
                continue

            try:
                aa = float(j)
                continue;
            except ValueError:
                aa=1

            count = count + 1
            content_result.append(0)
            content_result[count] = j # 각 내용으로부터 명사 저장


    content = data['Content']
    content_result_unique = list(set(content_result)) # 중복 제거

    N = len(content) # 총 문서의 수

    def tf(t, d):
        return d.count(t)

    def idf(t,docs):
        df = 0
        for doc in docs:
            #print(doc)
            df += t in doc
        return log(N/(df + 1))

    def tfidf(t, d,docs):
        return tf(t,d)* idf(t,docs)


    result = []
    for i in range(N): # 각 문서에 대해서 아래 명령을 수행

        result.append([])
        d = content[i]

        for j in range(len(content_result_unique)):
            t = content_result_unique[j]        
            result[-1].append(tf(t, d))

    tf_ = pd.DataFrame(result, columns = content_result_unique)

    result = []
    print(content)
    for j in range(len(content_result_unique)):
        t = content_result_unique[j]
        result.append(idf(t,content))

    idf_ = pd.DataFrame(result, index = content_result_unique, columns = ["IDF"])
    idf_ = idf_.sort_values(by=['IDF'], axis=0, ascending=True)

    idf_.to_excel("./단어중요도(IDF)_" + k + "_결과.xlsx")
 
    d = {}
    count = -1
    for i in idf_.index:
        count = count + 1
        d[i] = idf_['IDF'][count] * -1 
        if( d[i] > 0  ):
            d[i] = -0.001
  

    #plt.rc('font', family='NanumGothic') # For Windows
    wordcloud = WordCloud(font_path='malgun.ttf',
                        background_color="white",scale=3)
    wordcloud.generate_from_frequencies(frequencies=d)
    #plt.figure(figsize=(10,10),dpi=300)
    plt.figure(figsize = (20,10), dpi = 600)
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.tight_layout(pad=0)
    #plt.show()
    plt.savefig("./"+k+"_워드클라우드.png",width = 1600, height = 800, dpi = 300)
    